In [1]:
!pip install accelerate --q

^C


In [ ]:
!pip install pypdf --q
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off!pip install -Uqqq pip --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import HuggingFaceHub

loader = PyPDFLoader("CV_data.pdf")

docs = loader.load()
len(docs)

38

The Markdown file we're loading is the CV data paper. Let's see how we can use the RecursiveCharacterTextSplitter to split the document into smaller chunks:

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

82

Splitting the document into chunks is required due to the limited number of tokens a LLM can look at once (4096 for Llama 2). Next, we'll use the HuggingFaceEmbeddings class to create embeddings for the chunks:

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

1024


We'll use Chroma database to store/cache the embeddings and make it easy to search them

In [4]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("Transformer models", k=2)
print(results[0].page_content)

articles, book chapters, book reviews, and select commentary. After adding your publications, you need to focus on conferences and workshops.


In [5]:
# Assuming there's a configuration object or function that accepts backend settings
config = {
    # Other configuration settings
    "disable_exllama": True
}


In [9]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="cuda"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


In [10]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
 
template = """
<s>[INST] <<SYS>>
Act as a ML expert. Use the following information to answer the question at the end.
<</SYS>>
 
{context}
 
{question} [/INST]
"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
 
 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
 
result = qa_chain(
    "What topics should I include in my CV?"
)
print(result["result"].strip())

As a machine learning expert, I would recommend including the following topics in your CV:
1. Academic Qualifications: List all your relevant academic qualifications, including degrees, institutions, and graduation dates. Highlight any notable achievements or awards received during your studies.
2. Research Experience: Include any research experiences you have had, such as research assistant positions, internships, or writing projects. Provide detailed information about each project, including the topic, methodology, and outcomes.
3. Professional Experience: List any professional experience you have gained, such as internships, freelance work, or full-time jobs. Highlight any relevant skills or software proficiencies you possess, such as programming languages like Python or SQL, data analysis tools like Excel or Tableau, or machine learning frameworks like TensorFlow or PyTorch.
4. Research Interests: Outline your current research interests and areas of focus. This can include specific

In [11]:
from textwrap import fill
 
result = qa_chain(
    "Summarize the 3 most important topics of the CV."
)
print(fill(result["result"].strip(), width=80))

The 3 most important topics of a CV are: 1. Personal details - This section
includes your name, address, phone number, professional email, and visa status
(if relevant). It is essential to provide accurate and up-to-date contact
information to make it easy for potential employers to reach out to you. 2.
Career summary - This section should be around 5-7 sentences summarizing your
expertise in your disciplines, years of experience, notable research findings,
key achievements, and publications. It provides an overview of your career path
and helps recruiters quickly determine if you fit the job requirements. 3.
Education - In this section, provide an overview of your education history
starting from your most recent academic degree obtained (in reverse
chronological order). Highlight any relevant courses, certifications, or
training programs that align with the job requirements.
